In [1]:
import pandas as pd
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
ROOT_DATA_DIR = '../raw_data/Out_Feature_CSVs'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path) if l.split(".")[-1]=="csv"]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))  

Found 65 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Bad Directory
Found 65 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Good Directory
Number of csvs : 130
Number of Labels : 130
Found 17 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Bad Directory
Found 17 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Good Directory
Number of csvs : 34
Number of Labels : 34


In [3]:
train_data.head()

,csv_paths,csv_files,labels
0,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9364_03.csv,0
1,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9444_01.csv,0
2,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,74.csv,0
3,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9347_02.csv,0
4,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9336_01.csv,0


In [4]:
#create y_train and y_test as float64 type
y_train = train_data['labels']/1.0
y_test = test_data['labels']/1.0

In [5]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(0, inplace=True)
    list_of_lists = df.iloc[:,1:].values.tolist()
    return list_of_lists

def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [6]:
#create X_train and X_test
X_train = get_x(train_data)
X_test = get_x(test_data)

In [7]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=-1000)
print(X_train_pad.shape)

#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=-1000, maxlen=X_train_pad.shape[1])
print(X_test_pad.shape)

(130, 175, 2048)
(34, 175, 2048)


In [8]:
def build_model(input_shape):

  model_LSTM = Sequential()

  model_LSTM.add(layers.Masking(mask_value=-1000, input_shape=input_shape))
  model_LSTM.add(layers.LSTM(64, activation='tanh', return_sequences=True))
  model_LSTM.add(layers.Dropout(0.2))
  model_LSTM.add(layers.LSTM(32, activation='tanh'))
  model_LSTM.add(layers.Dense(8, activation='relu'))
  model_LSTM.add(layers.Dense(4, activation='relu'))
  model_LSTM.add(layers.Dense(1, activation='sigmoid'))

  model_LSTM.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model_LSTM

  print(model_LSTM.summary())

model = build_model(input_shape=(X_train_pad.shape[1], X_train_pad.shape[2]))

In [9]:
#Fitting the model to the train set
es = EarlyStopping(patience=100)

model.fit(X_train_pad, y_train, 
          epochs=1750, 
          batch_size=8, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,
          shuffle=True)

Epoch 1/1750
13/13 [==============================] - 15s 636ms/step - loss: 0.6745 - binary_accuracy: 0.6181 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8911 - val_binary_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/1750
13/13 [==============================] - 4s 332ms/step - loss: 0.6537 - binary_accuracy: 0.6442 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9893 - val_binary_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/1750
13/13 [==============================] - 4s 339ms/step - loss: 0.6586 - binary_accuracy: 0.6139 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.0792 - val_binary_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/1750
13/13 [==============================] - 4s 331ms/step - loss: 0.6502 - binary_accuracy: 0.6194 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1256 - val_binary_accuracy: 0.0000e+00 - val_precis

model.save("my_model")
model.save_weights("weights.h5")

In [10]:
#Evaluate the model on the train set
model.evaluate(X_train_pad, y_train)

5/5 [==============================] - 1s 215ms/step - loss: 1.1060 - binary_accuracy: 0.9462 - precision: 1.0000 - recall: 0.8923


[1.106017827987671, 0.9461538195610046, 1.0, 0.892307698726654]

In [11]:
#Evaluate the model on the test set
model.evaluate(X_test_pad, y_test)

2/2 [==============================] - 0s 61ms/step - loss: 4.9007 - binary_accuracy: 0.6471 - precision: 0.6923 - recall: 0.5294


[4.900697231292725, 0.6470588445663452, 0.692307710647583, 0.529411792755127]

In [12]:
#predictions of the test set compared to the actuals
model.predict(X_test_pad)

array([[3.7534251e-10],
       [4.9826770e-10],
       [4.0444950e-10],
       [8.8692296e-01],
       [9.1654164e-01],
       [3.9266743e-10],
       [9.1654164e-01],
       [3.7250539e-10],
       [9.5891953e-04],
       [7.3855527e-10],
       [4.1249737e-10],
       [4.0279127e-10],
       [9.1654164e-01],
       [6.0222355e-10],
       [6.7143308e-10],
       [1.5096875e-09],
       [5.3026766e-10],
       [9.1654164e-01],
       [5.0497091e-09],
       [3.0936214e-09],
       [9.1654164e-01],
       [1.8534090e-05],
       [9.1654164e-01],
       [9.1654164e-01],
       [9.1654164e-01],
       [4.0378123e-10],
       [4.2068449e-10],
       [3.9142761e-10],
       [9.1654164e-01],
       [1.5284333e-09],
       [9.1654164e-01],
       [9.1654164e-01],
       [4.9069437e-10],
       [9.1654164e-01]], dtype=float32)

In [13]:
y_test

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    1.0
33    1.0
Name: labels, dtype: float64